##  Implementation of Decision Trees 


In [83]:
class DecisionNode:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

#  each node of tree at max has four instances 

In [9]:
#importing datasets and other requirements
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [10]:
#loading the dataset
iris = load_iris()

In [22]:
x=iris.data
y=iris.target
data=np.c_[x,y]

In [12]:
##Question_class
header=iris['feature_names']+['target']
class Question:
    #initialises the column and value i.e.
    #is sepal length >=1 cm in this sepal leangth =col , 1cm=value
    def __init__(self,column,value):
        self.column=column
        self.value=value
    #it matches wheter the given data is in accordance with the value set or not
    #returns true or false correspondingly
    def match(self,data):
        value=data[self.column]
        return value>=self.value
    # This is just a helper method to print
    # the question in a readable format.
    def __repr__(self):
        condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
            

In [20]:
'demo of question class'
#forming of question
print(Question(0,5))
#to check wheter certain data point is in accordance with the quesiton of not
q=Question(0,5)
#4th data point
#checks wheter sepal length of 4th data point is >=5 or not 
print(q.match(x[3]))

Is sepal length (cm) >= 5?
False


In [21]:
#count the unique number of labels and stores it in  a dictionary
def count_values(rows):
    counts={}
    for row in rows:
        label=row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts


In [25]:
'demo of count_values'
#this shows our output labels are of three types(classses)->0,1,2
#each class having 50 entries
print(count_values(data))

{0.0: 50, 1.0: 50, 2.0: 50}


In [36]:
#SPLITS THE DATA BASED ON THE QUESTION
def partition(data,question):
    true_rows,false_rows=[],[]
    for row in data:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [37]:
'demo partition function'
#for each row in 0th col. it compares wheter value >=5 or not and correspondingle
#stores the value in true branch and false branch
true,false=partition(data,Question(0,5))
print(true)

[array([5.1, 3.5, 1.4, 0.2, 0. ]), array([5. , 3.6, 1.4, 0.2, 0. ]), array([5.4, 3.9, 1.7, 0.4, 0. ]), array([5. , 3.4, 1.5, 0.2, 0. ]), array([5.4, 3.7, 1.5, 0.2, 0. ]), array([5.8, 4. , 1.2, 0.2, 0. ]), array([5.7, 4.4, 1.5, 0.4, 0. ]), array([5.4, 3.9, 1.3, 0.4, 0. ]), array([5.1, 3.5, 1.4, 0.3, 0. ]), array([5.7, 3.8, 1.7, 0.3, 0. ]), array([5.1, 3.8, 1.5, 0.3, 0. ]), array([5.4, 3.4, 1.7, 0.2, 0. ]), array([5.1, 3.7, 1.5, 0.4, 0. ]), array([5.1, 3.3, 1.7, 0.5, 0. ]), array([5. , 3. , 1.6, 0.2, 0. ]), array([5. , 3.4, 1.6, 0.4, 0. ]), array([5.2, 3.5, 1.5, 0.2, 0. ]), array([5.2, 3.4, 1.4, 0.2, 0. ]), array([5.4, 3.4, 1.5, 0.4, 0. ]), array([5.2, 4.1, 1.5, 0.1, 0. ]), array([5.5, 4.2, 1.4, 0.2, 0. ]), array([5. , 3.2, 1.2, 0.2, 0. ]), array([5.5, 3.5, 1.3, 0.2, 0. ]), array([5.1, 3.4, 1.5, 0.2, 0. ]), array([5. , 3.5, 1.3, 0.3, 0. ]), array([5. , 3.5, 1.6, 0.6, 0. ]), array([5.1, 3.8, 1.9, 0.4, 0. ]), array([5.1, 3.8, 1.6, 0.2, 0. ]), array([5.3, 3.7, 1.5, 0.2, 0. ]), array([5. , 3

# Gini Impurity 

In [49]:
def gini(rows):
        count=count_values(rows)
        impurity=1
        for label in count:
            probab_of_label=count[label]/float(len(rows))
            impurity-=probab_of_label**2
        return impurity

#  Entropy

In [39]:
#entropy is the sum of p(x)log(p(x)) across all
#the diffrent possible results

In [41]:
def entropy(rows):
    entropy=0
    from math import log
    log2=lambda x:log(x)/log(2)
    count=count_values(rows)
    for label in count:
        p=count[label]/float(len(rows))
        entropy-=p*log2(p)
    return entropy

In [53]:
#weighted info gain
def info_gain_gini(current,left,right):
    p =float(len(left))/len(left)+len(right)
    return current-p*gini(left)-(1-p)*gini(right)


In [54]:
#weighted info gain
def info_gain_entropy(current,left,right):
    p =float(len(left))/len(left)+len(right)
    return current-p*entropy(left)-(1-p)*entropy(right)


#  Best Split

In [61]:
def best_split(rows):
    best_gain=0
    best_question=None
    current=gini(rows)
    features=len(rows[0])-1
    for col in range(features):
        #collects all unique classes for a feature
        values=set([row[col] for row in rows])
        for val in values:
            #for each class ask the question:
            question=Question(col,val)
            #devide the data
            true_rows,false_rows=partition(rows,question)
            if len(true_rows)==0 or len(false_rows) ==0:
                #no use go to next iteration
                continue
            gain=info_gain_gini(current,true_rows,false_rows)
            if gain>=best_gain:
                best_gain,best_question=gain,question
    return best_gain,best_question

In [110]:
a,b=best_split(data)


In [64]:
#leaf class which stores end result if no other children i.e node is a leaf
class Leaf:
    def __init__(self,rows):
        self.predictions=count_values(rows)

In [81]:
def build_tree(rows):
    #patition the data and return the respective highest gain and best quesiton
    gain, question = best_split(rows)

    # Base case: no further info gain
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    #returns the root question node storing branches as well as the quesiton
    return DecisionNode(question, true_branch, false_branch)

In [106]:
tree=build_tree(data)

In [100]:
def print_tree(node,indentation=""):
    '''printing function'''
    #base case means we have reached the leaf
    #if the node object is of leaf type
    if isinstance(node,Leaf):
        print(indentation+"PREDICTION",node.predictions)
        return 
    #print the question at node
    print(indentation + str(node.question))
    
    #call the function on true branch 
    print(indentation+ "True Branch")
    print_tree(node.true_branch,indentation + " ")
    
    #on flase branch
    print(indentation+ "False Branch")
    print_tree(node.false_branch,indentation + " ")

In [101]:
print_tree(tree)

Is petal length (cm) >= 6.9?
True Branch
 PREDICTION {2.0: 1}
False Branch
 Is sepal width (cm) >= 4.4?
 True Branch
  PREDICTION {0.0: 1}
 False Branch
  Is sepal width (cm) >= 4.2?
  True Branch
   PREDICTION {0.0: 1}
  False Branch
   Is sepal length (cm) >= 7.9?
   True Branch
    PREDICTION {2.0: 1}
   False Branch
    Is sepal width (cm) >= 4.1?
    True Branch
     PREDICTION {0.0: 1}
    False Branch
     Is sepal width (cm) >= 4.0?
     True Branch
      PREDICTION {0.0: 1}
     False Branch
      Is petal length (cm) >= 6.7?
      True Branch
       PREDICTION {2.0: 2}
      False Branch
       Is petal length (cm) >= 6.6?
       True Branch
        PREDICTION {2.0: 1}
       False Branch
        Is petal length (cm) >= 6.3?
        True Branch
         PREDICTION {2.0: 1}
        False Branch
         Is sepal length (cm) >= 7.7?
         True Branch
          PREDICTION {2.0: 1}
         False Branch
          Is sepal length (cm) >= 7.4?
          True Branch
           PR

# Output Corresponding To Project Requirements

In [287]:
def split_info(left,right):
    num=float(len(left))
    den=len(left)+len(right)
    p=num/den
    from math import log
    log10=lambda x:log(x)/log(10)
    return -(log10(p)*p+(1-p)*log10(1-p))

In [288]:
def print_tree_output(data,level):
    gain,question=best_split(data)
    #base case means we have reached the leaf
    #if the node object is of leaf type
    if gain==0:
        print("Level ",level)
        count=count_values(data)
        for value in count:
            print("Count of ",value," = ",count[value])
        print("Current entropy is = ",entropy(data))
        print("Reached Leaf Node") 
        return 
    print('Level ',level)
    count=count_values(data)
    for value in count:
        print("Count of ",value," = ",count[value])
    feature=question.column
    true_rows, false_rows = partition(data, question)
    split=split_info(true_rows,false_rows)
    gain1=info_gain_entropy(entropy(data),true_rows,false_rows)
    print("Current entropy is = ",entropy(data))
    print("Spliting on feature ",header[feature]," with gain ratio ",gain1/split)
    
    print_tree_output(true_rows,level+1)
    print_tree_output(false_rows,level+1)

In [289]:
print_tree_output(data,0)

Level  0
Count of  0.0  =  50
Count of  1.0  =  50
Count of  2.0  =  50
Current entropy is =  1.584962500721156
Spliting on feature  petal length (cm)  with gain ratio  13668.490189622818
Level  1
Count of  2.0  =  1
Current entropy is =  0.0
Reached Leaf Node
Level  1
Count of  0.0  =  50
Count of  1.0  =  50
Count of  2.0  =  49
Current entropy is =  1.5848973705351974
Spliting on feature  sepal width (cm)  with gain ratio  13501.92708309278
Level  2
Count of  0.0  =  1
Current entropy is =  0.0
Reached Leaf Node
Level  2
Count of  0.0  =  49
Count of  1.0  =  50
Count of  2.0  =  49
Current entropy is =  1.5848967831152563
Spliting on feature  sepal width (cm)  with gain ratio  13335.193479394704
Level  3
Count of  0.0  =  1
Current entropy is =  0.0
Reached Leaf Node
Level  3
Count of  0.0  =  48
Count of  1.0  =  50
Count of  2.0  =  49
Current entropy is =  1.5847621959588345
Spliting on feature  sepal length (cm)  with gain ratio  13169.966225081154
Level  4
Count of  2.0  =  1


Count of  1.0  =  1
Current entropy is =  0.0
Reached Leaf Node
Level  52
Count of  0.0  =  18
Count of  1.0  =  24
Current entropy is =  0.9852281360342516
Spliting on feature  petal width (cm)  with gain ratio  489.4717924904682
Level  53
Count of  1.0  =  2
Current entropy is =  0.0
Reached Leaf Node
Level  53
Count of  0.0  =  18
Count of  1.0  =  22
Current entropy is =  0.9927744539878084
Spliting on feature  petal width (cm)  with gain ratio  784.4115948839635
Level  54
Count of  1.0  =  1
Current entropy is =  0.0
Reached Leaf Node
Level  54
Count of  0.0  =  18
Count of  1.0  =  21
Current entropy is =  0.9957274520849256
Spliting on feature  petal length (cm)  with gain ratio  751.5167417736377
Level  55
Count of  1.0  =  1
Current entropy is =  0.0
Reached Leaf Node
Level  55
Count of  0.0  =  18
Count of  1.0  =  20
Current entropy is =  0.9980008838722996
Spliting on feature  petal length (cm)  with gain ratio  718.6047122691367
Level  56
Count of  1.0  =  1
Current entrop